In [1]:
from pyspark.sql import SparkSession, functions as F

In [2]:
import findspark
import warnings
warnings.filterwarnings('ignore')
from elasticsearch import Elasticsearch, helpers
import time
from pyspark.sql.functions import *

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark Elasticsearch")
    .master("local[2]")
    .config("spark.driver.memory","2048m")
    .config("spark.sql.shuffle.partitions", 4)
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1") 
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c5d9a4b7-afcf-46fe-a4ca-eb4f9f98990f;1.0
	confs: [default]
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.12.1 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in local-m2-cache
	found commons-logging#commons-logging;1.1.1 in local-m2-cache
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in local-m2-cache
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
:: resolution report :: resolve 861ms :: artifacts dl 7ms
	:: modules in use:
	com.google.protobuf#protobuf-java;2.5.0 from local-m2-cache in [default]
	commons-logging#commons-logging;1.1.1 from local-m2-cache in [default]
	javax.xml.bind#jaxb-api;2.3.1 from central in [default]
	org.ap

In [6]:
! wget -O ~/datasets/market1mil.csv.gz \
https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz

--2023-03-21 01:28:06--  https://github.com/erkansirin78/datasets/raw/master/market1mil.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/market1mil.csv.gz [following]
--2023-03-21 01:28:06--  https://raw.githubusercontent.com/erkansirin78/datasets/master/market1mil.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44525776 (42M) [application/octet-stream]
Saving to: ‘/home/train/datasets/market1mil.csv.gz’

100%[======================================>] 44,525,776  1.97MB/s   in 59s    

2023-03-21 01:29:05 (740 KB/s) - ‘/home/train/datasets/market1mil.csv.

In [4]:
df = spark.read.format("csv") \
.option("header", True) \
.option("sep",";") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/market1mil.csv")

In [5]:
df.limit(5).toPandas()

2023-03-21 22:36:50,312 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,LOGICALREF,COUNT_,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,...,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,SPECODE,CAPIBLOCK_CREADEDDATE
0,1,1,8,TOZ SEKER,15560,2.01.2017 00:00,45,"2,65","5,3","4,9",...,Hanım CANBULAT,None,None,İÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
1,2,1,20868,KIRMIZI MERCIMEK,15560,2.01.2017 00:00,"1,006","2,8","2,82","2,79",...,Hanım CANBULAT,167,BAKLİYAT,GIDA,BAKLİYAT,AÇIK BAKLİYAT,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
2,3,1,8583,"TEST MATIK 1,5 KG NORMAL",15560,2.01.2017 00:00,1,"4,95","4,95","4,19",...,Hanım CANBULAT,229,TEST,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,TOZ DETERJAN,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
3,4,1,1454,BIZIM MAKARNA BONCUK,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",...,Hanım CANBULAT,146,ÜLKER,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50
4,5,1,13519,FILIZ MAKARNA KISA KESME 500 GR,15560,2.01.2017 00:00,1,"1,1","1,1","1,02",...,Hanım CANBULAT,52,FİLİZ,GIDA,MAKARNA,MAKARNA,3.01.2017 09:25,3.01.2017 09:25,K,14.07.2018 01:50


In [6]:
df.count()

999853

In [7]:
df.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- COUNT_: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- FICHENO: integer (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- LINENET: string (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (

In [8]:
df1 = df.dropna()

In [9]:
df1.count()

597075

In [10]:
df2=df1.withColumn("STARTDATE",
                     F.unix_timestamp(F.to_timestamp("STARTDATE","d.M.y H:m")) *1000)\
.withColumn("AMOUNT", regexp_replace(col("AMOUNT"), ",", ".")) \
.withColumn("PRICE", regexp_replace(col("PRICE"), ",", ".")) \
.withColumn("LINENETTOTAL",regexp_replace("LINENETTOTAL",",","."))


In [11]:
df2.limit(5).toPandas()

,LOGICALREF,COUNT_,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,...,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,SPECODE,CAPIBLOCK_CREADEDDATE
0,2,1,20868,KIRMIZI MERCIMEK,15560,2.01.2017 00:00,1.006,2.8,2.82,"2,79",...,Hanım CANBULAT,167,BAKLİYAT,GIDA,BAKLİYAT,AÇIK BAKLİYAT,1483424700000,3.01.2017 09:25,K,14.07.2018 01:50
1,3,1,8583,"TEST MATIK 1,5 KG NORMAL",15560,2.01.2017 00:00,1,4.95,4.95,"4,19",...,Hanım CANBULAT,229,TEST,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,TOZ DETERJAN,1483424700000,3.01.2017 09:25,K,14.07.2018 01:50
2,4,1,1454,BIZIM MAKARNA BONCUK,15560,2.01.2017 00:00,1,1.1,1.1,"1,02",...,Hanım CANBULAT,146,ÜLKER,GIDA,MAKARNA,MAKARNA,1483424700000,3.01.2017 09:25,K,14.07.2018 01:50
3,5,1,13519,FILIZ MAKARNA KISA KESME 500 GR,15560,2.01.2017 00:00,1,1.1,1.1,"1,02",...,Hanım CANBULAT,52,FİLİZ,GIDA,MAKARNA,MAKARNA,1483424700000,3.01.2017 09:25,K,14.07.2018 01:50
4,6,1,8639,BINGO ULTRA CAM.SUYU 750 ML KLASIK,15560,2.01.2017 00:00,1,2.45,2.45,"2,08",...,Hanım CANBULAT,224,BİNGO,DETERJAN TEMİZLİK,ÇAMAŞIR YIKAMA,ÇAMAŞIR SULARI,1483424700000,3.01.2017 09:25,K,14.07.2018 01:50


In [12]:
df1.selectExpr("MAX(STARTDATE)", "MIN(STARTDATE)").limit(30).toPandas()

,max(STARTDATE),min(STARTDATE)
0,9.05.2017 21:18,1.02.2017 08:36


In [13]:
df2.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- COUNT_: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- FICHENO: integer (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- LINENET: string (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: long (nu

In [38]:

df3 = df2.withColumn("LONGITUDE", regexp_replace(col("LONGITUDE"), ",", "."))
df4 = df3.withColumn("LATITUDE", regexp_replace(col("LATITUDE"), ",", ".")) \
.withColumn("LONGITUDE", expr("substr(LONGITUDE, 1, 5)")) \
.withColumn("LATITUDE", expr("substr(LATITUDE, 1, 5)")) \
.withColumn("LONGITUDE", col("LONGITUDE").cast("float")) \
.withColumn("LATITUDE", col("LATITUDE").cast("float")) 


In [40]:
df4 = df4.withColumn("location", concat(col("latitude"), lit(","), col("longitude"))) \
.withColumn("LOCATION", regexp_replace("LOCATION", "[a-zA-Z]", "")) \
.withColumn("LOCATION", col("LOCATION").cast("float")) 


In [41]:
df4.printSchema()

root
 |-- LOGICALREF: string (nullable = true)
 |-- COUNT_: integer (nullable = true)
 |-- ITEMCODE: integer (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- FICHENO: integer (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: string (nullable = true)
 |-- PRICE: string (nullable = true)
 |-- LINENETTOTAL: string (nullable = true)
 |-- LINENET: string (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- SALESMAN: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- LATITUDE: float (nullable = true)
 |-- LONGITUDE: float (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- CLIENTNAME: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: long (null

In [42]:
market1mil_index = {
  "mappings": {
    "properties": {
      "LOGICALREF": {"type": "integer"},
      "ITEMCODE": {"type": "integer"},
      "ITEMNAME": {"type": "text"},
      "AMOUNT": {"type": "float"},
      "PRICE": {"type": "float"},
      "LINENETTOTAL": {"type": "float"},
      "BRANCH": {"type": "keyword"},
      "CITY": {"type": "keyword"},
      "BRAND": {"type": "keyword"},
      "STARTDATE": {"type": "date"},
      "LOCATION": {"type": "geo_point"}
    }
  }
}

In [43]:
es = Elasticsearch(["localhost:9200"])

In [44]:
try:
    es.indices.delete("online-market-spark")
    print("online-market-spark  index deleted.")
except:
    print("No index")

online-market-spark  index deleted.


In [45]:
es.indices.create("online-market-spark", body=market1mil_index)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'online-market-spark'}

In [46]:
start_time = time.time()

df4.write \
    .format("org.elasticsearch.spark.sql") \
    .mode("overwrite") \
    .option("es.nodes", "localhost") \
    .option("es.port","9200") \
    .save("online-market-spark")


print("----- %s secs -----" %(time.time() - start_time))

----- 116.37668108940125 secs -----


In [49]:
df_es = spark.read \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "localhost") \
    .option("es.port","9200") \
    .load("online-market-spark")

In [50]:
df_es.limit(5).toPandas()

,AMOUNT,BRANCH,BRANCHNR,BRAND,BRANDCODE,CAPIBLOCK_CREADEDDATE,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,CITY,...,LINENET,LINENETTOTAL,LOCATION,LOGICALREF,LONGITUDE,PRICE,REGION,SALESMAN,SPECODE,STARTDATE
0,1.0,İstanbul Subesi,40,KAR,74,14.07.2018 02:17,GIDA,BÜSKİVİ ÇEREZ,CİPS,İstanbul,...,"1,86",2.00,None,3296,28.969999,2.00,Marmara,İsmet SARTIK,K,2017-01-06 19:40:00
1,1.0,İstanbul Subesi,40,COCO COLA,30,14.07.2018 02:17,İÇECEK,GAZLI İÇECEK,COLA,İstanbul,...,"2,27",2.45,None,3297,28.969999,2.45,Marmara,İsmet SARTIK,K,2017-01-06 19:40:00
2,1.0,İstanbul Subesi,40,COCO COLA,30,14.07.2018 02:17,İÇECEK,GAZLI İÇECEK,COLA,İstanbul,...,"2,27",2.45,None,3298,28.969999,2.45,Marmara,İsmet SARTIK,K,2017-01-06 19:40:00
3,1.0,İstanbul Subesi,40,ÜLKER,146,14.07.2018 02:17,GIDA,BÜSKİVİ ÇEREZ,KEK,İstanbul,...,"2,77",2.99,None,3299,28.969999,2.99,Marmara,İsmet SARTIK,K,2017-01-06 19:40:00
4,1.0,İstanbul Subesi,40,KEMAL KÜKRER,169,14.07.2018 02:17,GIDA,HAZIR YEMEKLER,SİRKE SOS,İstanbul,...,"1,48",1.60,None,3301,28.969999,1.60,Marmara,İsmet SARTIK,K,2017-01-06 19:40:00


In [ ]:
df_es.count()